In [16]:
import pandas as pd
import numpy as np
import sys
import gc
import time
from sklearn.ensemble import RandomForestClassifier

from keras.models import Sequential, model_from_json
from keras.layers.core import Dense, Activation
from keras.optimizers import Adagrad

import utils
from grid import RoadGrid

In [30]:
tr_names = ['forward0', 'forward1', 'forward2', 'forward3', 'forwardbackward4', 'forwardbackward5', 'bu_1', 'bu_2', 'backward1', 'backward3']
te_names = ['backward2']

tr_data = utils.get_4g_data(tr_names)
te_data = utils.get_4g_data(te_names)

eng_para = utils.get_4g_engpara()
tr_time, tr_feature, tr_label_ = utils.make_rf_dataset(tr_data, eng_para)


tr_label = tr_data[['Longitude', 'Latitude']].values
te_label = te_data[['Longitude', 'Latitude']].values
#rg = RoadGrid(tr_label, 30)
#tr_label_ = np.array(rg.transform(tr_label, False))


In [33]:
error = []
header = ['RNCID', 'Test', 'Train', 'Time', 'mean', 'median', '80%', '90%']
exp_log = []
for record in te_data.groupby(['RNCID_1']):
    content = [record[0], len(record[1]), len(tr_data[tr_data.RNCID_1 == record[0]])]
    start = time.time()
    mask = (tr_data.RNCID_1.values == record[0])

    tr_feature_ = tr_feature[mask]
    tr_label_ = tr_label[mask]
    rg = RoadGrid(tr_label, 30)
    tr_label_ = np.array(rg.transform(tr_label_, False))
    
    te_time_, te_feature_, te_label_ = utils.make_rf_dataset(record[1], eng_para)
    te_feature_ = te_feature_.values
    te_label_ = te_label_.values
    
    est = RandomForestClassifier(
        n_jobs=-1,
        n_estimators = 50,
        max_features='sqrt',
#        bootstrap=True,
        criterion='entropy'
    ).fit(tr_feature_, tr_label_)
    
    te_pred = est.predict(te_feature_)
    te_pred = np.array([rg.grid_center[gid] for gid in te_pred])
    
    error_ = [utils.distance(pt1, pt2) for pt1, pt2 in zip(te_pred, te_label_)]
    error += error_
    content += [time.time()-start, np.mean(error_), np.median(error_), np.percentile(error_, 80), np.percentile(error_, 90)]
    exp_log.append(content)
    del est
    gc.collect()

In [26]:
utils.report(error, '%d\t%d\t%d' % (len(tr_data), len(te_data), tr_feature.shape[1]))

56949	4675	36	51.19	28.10	45.40	68.90	117.80


In [32]:
utils.report(error, '%d\t%d\t%d' % (len(tr_data), len(te_data), tr_feature.shape[1]))

69210	4675	36	46.63	28.20	43.90	63.80	104.90


In [35]:
pd.DataFrame(exp_log, columns=header)

,RNCID,Test,Train,Time,mean,median,80%,90%
0,99077,23,202,0.718829,25.378261,11.60,46.98,58.94
1,99078,123,471,0.631705,71.454472,62.20,93.90,130.04
2,99084,74,1805,0.860292,32.017568,21.70,52.24,62.44
3,99096,282,4672,1.167534,72.805319,26.15,98.30,248.92
4,99097,31,453,0.590877,33.680645,19.10,20.60,23.00
5,99099,79,950,0.661553,33.888608,15.00,45.62,87.76
6,99110,38,805,0.680425,28.623684,17.30,32.64,56.87
7,99112,369,3905,1.028577,74.025474,41.30,104.92,169.54
8,99121,168,3266,1.037515,53.691071,27.35,104.02,149.96
9,99127,671,3063,0.966859,38.059463,25.10,52.50,80.80
